In [8]:
from src.models.our_model import OurModel
from src.models.pretrained_models import VGG16Pretrained, ResNetPretrained
import torch
from src.models.ensemble import HardVotingEnsemble, SoftVotingEnsemble, MetaClassifier, StackingEnsemble
from src.utils import load_data, evaluate_model
from src.model_trainer import ModelTrainer
from src.transformations import normalized_simple_transform, pretrained_transform
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
torch.manual_seed(123)
print(device)
torch.set_num_threads(14)

cpu


In [2]:
test_loader = load_data('./data/test', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)

In [ ]:
ourmodel = OurModel(aux_enabled=False, se_squeeze=8)  
ourmodel.load_state_dict(torch.load("./saved_models/ourmodel/combined_20/OurModel_20.pth", map_location=device))
ourmodel.to(device)

OurModel(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (inception1): Inception(
    (branch1): Sequential(
      (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch2): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (branch3): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(12, ep

In [4]:
evaluate_model(model=ourmodel, dataloader=test_loader, device=device)

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 

{'accuracy': 0.7336777777777778,
 'f1_score': 0.7325873347487415,
 'roc_auc': np.float64(0.9672593004801099)}

In [5]:
ourmodel = OurModel(aux_enabled=False, se_squeeze=8)  
ourmodel.load_state_dict(torch.load("./saved_models/ourmodel/combined_20/OurModel_16.pth", map_location=device))
ourmodel.to(device)

OurModel(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (inception1): Inception(
    (branch1): Sequential(
      (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (branch2): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (branch3): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(12, ep

In [6]:
evaluate_model(model=ourmodel, dataloader=test_loader, device=device)

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 

{'accuracy': 0.7467555555555555,
 'f1_score': 0.7449930276087703,
 'roc_auc': np.float64(0.9699651601508916)}

In [ ]:
test_loader_pretrained = load_data('./data/test', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)
vgg16 = VGG16Pretrained(device="cpu")  
vgg16.load_state_dict(torch.load("./saved_models/vgg_pretrained/regularization_params_001_2/VGG16Pretrained_10.pth", map_location=device))
vgg16.to(device)
evaluate_model(model=vgg16, dataloader=test_loader_pretrained, device=device)

c:\Users\karim\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\karim\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0 